In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import scipy.stats as stats
import datetime
from tqdm import tqdm
import random

mpl.rc('font', family='Malgun Gothic')

In [33]:
# generate dataframe
df_member = pd.read_csv('./Member_Data.csv', encoding='cp949')
df_member = df_member.drop(columns='Unnamed: 0')

df_product = pd.read_csv('./Product_Data.csv', encoding='utf-8')

df_sales = pd.read_csv('./Sales_Data05.csv', encoding='utf-8')

In [4]:
df_member.head()

,회원번호,회원상태,성별,나이,등록카드,결혼,구독여부,주소지,세부주소지
0,18764160,정상회원,여,68,농협중앙회,기혼,False,서울특별시,성동구
1,18792000,정상회원,남,83,연결앱결제,NaN,False,강원도,강릉시
2,18942336,정상회원,여,39,신한은행,기혼,False,인천광역시,중구
3,18949760,정상회원,여,73,기업은행,NaN,NaN,강원도,홍천군
4,19391488,정상회원,여,52,연결앱결제,기혼,False,대전광역시,중구


In [5]:
df_product.head()

,제품번호,물품명,물품대분류,물품중분류,상품중량
0,100021V2_0,2단무늬컵,식기/편백,자기,1p
1,100022V2_1,7곡딸기롤과자,과자,스낵,100g/10개입
2,100023V2_2,7곡참식,식사대용,선식/생식,700g
3,100024V2_3,가리비,생물수산,패류/갑각류,1.5kg
4,100025V2_3,가리비,생물수산,패류/갑각류,1kg


In [6]:
df_sales.head()

,회원번호,회원상태,구매수량,구매금액,주문일시,배송시작일,배송완료일,사용 적립금,사용 포인트 네이버,주문취소여부,주문시간,제품번호
0,1032097472,정상회원,1.0,7083,2021-01-02,NaN,NaN,0,0,주문취소,오후 12:60,100021783V2_1337
1,1032097472,정상회원,1.0,29865,2021-01-02,2021-01-02,2021-01-02,0,0,NaN,오후 12:60,100022137V2_1606
2,1032097472,정상회원,1.0,23164,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,100021452V2_1113
3,369152832,정상회원,1.0,16655,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,10002931V2_708
4,1032097472,정상회원,1.0,8423,2021-01-02,NaN,NaN,0,0,주문취소,오후 12:60,100022085V2_1559


## 고객 이탈률
df_member 에서 회원 상태로 정상회원과 탈퇴회원 파악 가능

고객 이탈률(%) = ( 탈퇴회원 수 / 전체 회원 수 ) * 100

In [7]:
df_member['회원상태'].value_counts(dropna=False)

회원상태
정상회원     12380
탈퇴         156
탈퇴처리중        2
NaN          1
탈퇴신청         1
Name: count, dtype: int64

In [8]:
# 탈퇴 회원 수
cond_withdraw = (df_member['회원상태'] != '정상회원')
withdraw = len(df_member.loc[cond_withdraw])

# 고객 이탈률 계신 탈퇴 회원 수
cond_withdraw = (df_member['회원상태'] != '정상회원')
withdraw = len(df_member.loc[cond_withdraw])

# 고객 이탈률 계신
withdraw_rate = (withdraw / len(df_member)) * 100
print(f"고객 이탈률(%): {np.round(withdraw_rate, 3)}%")

# 고객 이탈률은 1.276%로 크게 높다고 생각이 들지는 않음. 고객 이탈률을 유의미하게 줄이기보다
# 다양한 마케팅 전략을 시도하면서 고객 이탈률을 유지하거나 10% 내외로 변동하게 유지하는 것이 중요하다고 판단

# 탈퇴 회원의 특징은 무엇인지 그들의 df_sales에서 확인해봐도 좋을 듯!!!

고객 이탈률(%): 1.276%


## 구매주기 n개월 이하 고객 비율
df_sales에서 각 고객별로 주문 내역을 monthly로 살펴본 뒤, 구매주기가 n개월 이하인 고객 파악

n개월 구매주기 고객 비율(%) = ( n개월 구매주기 고객 수 / 전체 고객 수 ) * 100


In [9]:
# to_datetime
df_sales[['주문일시',
          '배송시작일',
          '배송완료일']] = df_sales[['주문일시', '배송시작일', '배송완료일']].apply(pd.to_datetime)

In [10]:
# check sales of each member according to orderdate
memberwise_sales = df_sales.groupby(['회원번호','주문일시']).count().reset_index()
memberwise_sales

,회원번호,주문일시,회원상태,구매수량,구매금액,배송시작일,배송완료일,사용 적립금,사용 포인트 네이버,주문취소여부,주문시간,제품번호
0,18764160,2021-01-03,2,2,2,2,2,2,2,0,2,2
1,18764160,2021-01-23,3,3,3,3,3,3,3,0,3,3
2,18764160,2021-03-30,2,2,2,2,2,2,2,0,2,2
3,18764160,2021-04-11,1,1,1,1,1,1,1,0,1,1
4,18764160,2021-04-14,1,1,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
193289,1670620864,2021-04-27,1,1,1,1,1,1,1,0,1,1
193290,1670620864,2021-04-28,8,8,8,8,8,8,8,0,8,8
193291,1670620864,2021-04-30,2,2,2,2,2,2,2,0,2,2
193292,1670620864,2021-06-02,2,2,2,2,2,2,2,0,2,2


In [11]:
def isloyal(member_list, memberwise_sales, gap):
    a = []
    b = []
    col_name = ['회원번호', 'count']
    
    # 멤버 한 명씩 주문 일시를 살펴볼 때,
    for member in member_list:
        target_member = memberwise_sales[memberwise_sales['회원번호'] == member].sort_values(by='주문일시')
        count = 0  # 멤버마다 구매유지를 하지 못한 횟수
        
        # 특정 타겟 멤버의 주문 내역을 순회
        for i in range(len(target_member) - 1):
            sales_gap = (target_member['주문일시'].iloc[i+1] - target_member['주문일시'].iloc[i])
            if sales_gap > gap:
                count += 1

        a.append(member)
        b.append(count)

    list_df = pd.DataFrame({'회원번호': a, 'count': b})
    
    return list_df

In [12]:
# 구매유지의 기준이 되는 timedelta
gap = datetime.timedelta(days=120)

# 실행
result = isloyal(memberwise_sales['회원번호'].unique(), memberwise_sales, gap)
print(result)

             회원번호  count
0        18764160      0
1        18792000      0
2        18942336      0
3        18949760      0
4        19391488      0
...           ...    ...
12535  1644957952      0
12536  1645334720      0
12537  1670400000      0
12538  1670606016      1
12539  1670620864      0

[12540 rows x 2 columns]


In [13]:
result['count'].value_counts()

count
0    11274
1     1262
2        4
Name: count, dtype: int64

In [14]:
# 장기고객의 기준이 되는 count는 임의로 1번이내로 설정. gap 값 변화해가면서 살펴볼 것!!!
print(f"장기고객비율: {result['count'].value_counts()[0] / 12540}")

장기고객비율: 0.899043062200957


## 배송 지연률
해당 기업이 식품을 당일 배송해주는 시스템으로 구성되었으므로, 배송 지연의 기준 일자는 2일로 설정

주문일시와 배송완료 2일일 이상 차이나는 경우를 배송지연이라정의

, 배송 지연율(%) = ( 배송지연 발생 건 / 전체 주문 건 ) * 100

In [15]:
# 멤버별로 배송시작일과 배송완료일
memberwise_delivery = df_sales.groupby(['회원번호','주문일시','배송시작일','배송완료일']).count().reset_index()
memberwise_delivery

,회원번호,주문일시,배송시작일,배송완료일,회원상태,구매수량,구매금액,사용 적립금,사용 포인트 네이버,주문취소여부,주문시간,제품번호
0,18764160,2021-01-03,2021-01-03,2021-01-03,1,1,1,1,1,0,1,1
1,18764160,2021-01-03,2021-01-05,2021-01-06,1,1,1,1,1,0,1,1
2,18764160,2021-01-23,2021-01-23,2021-01-25,1,1,1,1,1,0,1,1
3,18764160,2021-01-23,2021-01-24,2021-01-25,1,1,1,1,1,0,1,1
4,18764160,2021-01-23,2021-01-24,2021-01-26,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
424684,1670620864,2021-04-30,2021-04-30,2021-04-30,1,1,1,1,1,0,1,1
424685,1670620864,2021-04-30,2021-04-30,2021-05-01,1,1,1,1,1,0,1,1
424686,1670620864,2021-06-02,2021-06-02,2021-06-03,1,1,1,1,1,0,1,1
424687,1670620864,2021-06-02,2021-06-02,2021-06-04,1,1,1,1,1,0,1,1


In [16]:
def late_delivery(member_list, memberwise_sales, deli_gap):
    a = []
    b = []
    col_name = ['회원번호', 'delivery_count']
    
    # 멤버 한 명씩 주문 일시를 살펴볼 때,
    for member in member_list:
        target_member = memberwise_sales[memberwise_sales['회원번호'] == member].sort_values(by='주문일시')
        delivery_count = 0  # 멤버마다 배송지연이 발생한 횟수
        
        # 특정 타겟 멤버의 주문 내역을 순회
        for i in range(len(target_member)):
            delivery_gap = (target_member['배송완료일'].iloc[i] - target_member['주문일시'].iloc[i])
            if delivery_gap > deli_gap:
                delivery_count += 1

        a.append(member)
        b.append(delivery_count)

    list_df = pd.DataFrame({'회원번호': a, 'delivery_count': b})
    
    return list_df

In [17]:
# 배송지연의 기준이 되는 timedelta (2일)
deli_gap = datetime.timedelta(days=2)

# 실행
deli_result = late_delivery(memberwise_sales['회원번호'].unique(), memberwise_delivery, deli_gap)
print(deli_result)

             회원번호  delivery_count
0        18764160               4
1        18792000               0
2        18942336               6
3        18949760               0
4        19391488               0
...           ...             ...
12535  1644957952               1
12536  1645334720               0
12537  1670400000             128
12538  1670606016               0
12539  1670620864               1

[12540 rows x 2 columns]


In [18]:
deli_result['delivery_count'].value_counts()

delivery_count
0      5642
1      2347
2      1185
3       774
4       530
5       415
6       321
7       235
8       147
9       145
10      138
12      102
11      100
13       64
14       58
15       48
16       43
17       32
18       31
21       19
22       19
19       18
23       16
20       12
24       12
25       11
26       10
30        9
29        7
32        7
28        6
38        6
27        5
31        4
34        3
39        3
35        3
48        2
44        2
36        2
43        1
58        1
40        1
45        1
33        1
37        1
128       1
Name: count, dtype: int64

In [19]:
# 배송지연의 기준이 되는 deli_count는 임의로 5번이상으로 설정. deli_gap 값 변화해가면서 살펴보고
# 단순히 횟수가 아니라 개인의 전체 주문의 몇 % 인지로도 살펴보기!!!
print(f"배송지연비율: {(1 - (5642+2347+1185+774+530) / 12540) * 100}")

배송지연비율: 16.443381180223284


## 정기구독률
정기구독률 = (구독 회원수) / (전체 회원수) * 100

In [279]:
# 구독X = NaN(결측값), False  /  구독O =  True 가정.
df_member['구독여부'].value_counts(dropna=False)

구독여부
False    8441
nan      2346
True     1753
Name: count, dtype: int64

In [281]:
# 정기구독 회원 수
cond_sub = (df_member['구독여부'] == 'True')
sub_customer = len(df_member.loc[cond_sub])

# 정기 구독률
subscription_rate = (sub_customer / len(df_member)) * 100
print(f"정기구독률은 {subscription_rate}% 이다.")

정기구독률은 13.9792663476874% 이다.


## 유령회원 수
유령고객 = 가장 최신 주문 건 기준 n개월 간 주문이 없는 고객.

In [157]:
def isghost(member_list, memberwise_sales, active_member):
    a = []
    b = []
    col_name = ['회원번호', 'isghost']

    # 모든 정상회원 통틀어 가장 최근 주문 시기
    last_order = memberwise_sales['주문일시'].max() 

    chronologic_order = memberwise_sales.sort_values(by=['회원번호',
                                                         '주문일시'], ascending=[True, False])[['회원번호',
                                                                                            '주문일시']]
    
    # 멤버 한 명씩 주문 일시를 살펴볼 때, 회원별 주문일시를 최신순으로 정렬해서 순회
    for member in tqdm(member_list):
        isghost = 0
        
        target_member = chronologic_order[chronologic_order['회원번호'] == member]

        # 특정 타겟 멤버의 주문 내역을 순회, 가장 최근 주문과 현재시점 사이의 시간간격 계산
        
        latest_gap = last_order - target_member['주문일시'].iloc[0]

        if latest_gap > datetime.timedelta(days=120):
            #현재 시점으로부터 4개월 넘게 주문하지 않은 경우 유령회원으로 판별
            isghost = 1
            
        a.append(member)
        b.append(isghost)

    list_df = pd.DataFrame({'회원번호': a, 'isghost': b})
    
    return list_df

In [158]:
# 실행
active_member = df_member[df_member['회원상태'] == '정상회원']['회원번호']

active_memberwise_sales = memberwise_sales[memberwise_sales['회원번호'].isin(active_member)]

isghost_result = isghost(active_memberwise_sales['회원번호'].unique(), active_memberwise_sales, active_member)
print(isghost_result)

100%|██████████| 12380/12380 [00:09<00:00, 1310.95it/s]


             회원번호  isghost
0        18764160        0
1        18792000        1
2        18942336        0
3        18949760        0
4        19391488        1
...           ...      ...
12375  1644772352        0
12376  1644957952        0
12377  1645334720        0
12378  1670606016        0
12379  1670620864        1

[12380 rows x 2 columns]


In [159]:
isghost_result['isghost'].value_counts()

isghost
0    9522
1    2858
Name: count, dtype: int64

In [160]:
ghost_customer = isghost_result['isghost'].sum()
active_member = df_member[df_member['회원상태'] == '정상회원']['회원번호'].unique()

# 유령고객 비율
ghost_customer_rate = ghost_customer / len(active_member) * 100
print(f"유령고객 비율은 {ghost_customer_rate}% 이다.")

유령고객 비율은 23.08562197092084% 이다.


# Roadmap
1. 고객별 연령대, 거주지, 구매빈도, 구매금액, 구매제품종류(중분류, 소분류, 제품번호 등 다양한 범위에서) 등의 데이터에서 고객 그룹에 대한 특징을 파악[군집분석, RFM분석 등]하고 비슷한 고객 군집 생성해 고객 유형을 설정.

2. 인기상품이나 카테고리에 대한 소비 트렌드 확인(월별 인기상품, 분기별 인기 상품(카테고리) 등)

3. Sales 데이터를 활용하여 고객별로 구매주기가 어떤지 확인하고, 이를 이용해 다음 구매 주기에 어떤 상품을 구매할지 예측하는 모델 구성
[회원마다 과거 구매이력을 통해 평균 구매 주기를 계산하고, 다음 구매주기에 맞춰 상품을 추천할 수 있도록 정기구독 목록을 작성할 수 있도록. 추가로, 회원A와 비슷한 제품을 구매한 회원그룹B를 파악하고 A가 구매하지 않았지만 B가 구매한 기록이 있는 제품을 함께 추천할 수 있도록 모델 구성]

4. 주문시간, 거주지, 구매제품종류 등을 input으로 입력받으면 배송지연 여부를 예측하는 모델을 Sales 데이터를 활용하여 생성. 고객의 주문특성(주문시간, 거주지, 구매상품 종류 등)과 배송기간 사이의 관계를 회귀분석(다른 방법을 사용해도 무방)을 이용해 파악. 배송 지연된 경험을 한 고객들만 따로 뽑아서 재구매율과 고객이탈률, 구독유무를 계산(가설검정: 배송지연발생유무(혹은 빈도)가 재구매율, 이탈률, 구독유무와 상관관계가 있다)

5. 구매내역, 구매주기, 배송지연빈도(혹은 유무), 구독 유무들을 input으로 받아 특정 고객의 이탈이나 유령고객이 될 확률을 계산하는 모델 생성.

6. 구매주기, 배송지연빈도, 고객 이탈확률(유령고객 될 확률), 고객유형을 input으로 구독확률과 구매주기가 n개월 이하로 유지될 확률(자주 상품을 구매할 확률)을 계산하는 모델 생성.


# Feature 1.
- RFM 분석 및 고객 클러스터링

In [26]:
# 날짜 변환
latest_date = df_sales['주문일시'].max()  # 데이터 내 최신 주문일

# RFM분석
rfm = pd.pivot_table(df_sales, index="회원번호",values=["주문일시", "구매금액"],
    aggfunc={"주문일시": [lambda x: (latest_date - x.max()).days, "count"], "구매금액": "sum"}).reset_index()

# 컬럼 이름 변경
rfm.columns = ["회원번호", "Recency", "Frequency", "Monetary"]

# RFM 점수 (5단계 )
rfm["R_Score"] = pd.qcut(rfm["Recency"], 5, labels=[5, 4, 3, 2, 1])
rfm["F_Score"] = pd.qcut(rfm["Frequency"], 5, labels=[1, 2, 3, 4, 5])
rfm["M_Score"] = pd.qcut(rfm["Monetary"], 5, labels=[1, 2, 3, 4, 5])

# 최종 RFM 점수 계산
rfm["RFM_Score"] = rfm[["R_Score", "F_Score", "M_Score"]].astype(int).sum(axis=1)

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

rfm_file = "RFM_Score.csv"
rfm_df = pd.read_csv(rfm_file, encoding="utf-8")

rfm_features = rfm_df[["Recency", "Frequency", "Monetary"]]

# 데이터 정규화
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_features)

# 최적 K 찾기 (엘보우 방법)
distortions = []
K_range = range(2, 11)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(rfm_scaled)
    distortions.append(kmeans.inertia_)

# K-Means 클러스터링 (K=4 )
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
rfm_df["클러스터"] = kmeans.fit_predict(rfm_scaled)

# 클러스터별 평균 RFM 값 
cluster_analysis_ = rfm_df.groupby("클러스터")[["Recency", "Frequency", "Monetary"]].mean().reset_index()

cluster_analysis_

,클러스터,Recency,Frequency,Monetary
0,0,199.894821,6.494371,8.351010e+04
1,1,32.692089,31.096529,3.530412e+05
2,2,1.628415,583.907104,7.706210e+06
3,3,6.934623,200.198799,2.330087e+06


In [42]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# 컬럼 선택 
df_member["구독여부"] = df_member["구독여부"].astype(str)  # 타입을 문자열로 변환

# 데이터 변환
column_transformer = ColumnTransformer([('num', StandardScaler(), ["나이"]),
    ('cat', OneHotEncoder(handle_unknown='ignore', dtype=int), ["성별", "결혼", "구독여부"])])

# K-Means 클러스터링 (K=4)
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_member["클러스터"] = kmeans.fit_predict(column_transformer.fit_transform(df_member))

# Feature 3.
Sales 데이터를 활용하여 고객별로 구매주기가 어떤지 확인하고, 이를 이용해 다음 구매 주기에 어떤 상품을 구매할지 예측하는 모델 구성 

회원마다 과거 구매이력을 통해 평균 구매 주기를 계산하고, 다음 구매주기에 맞춰 상품을 추천할 수 있도록 정기구독 목록을 작성할 수 있어야함.

회원A와 비슷한 제품을 구매한 회원그룹B를 파악하고 A가 구매하지 않았지만 B가 구매한 기록이 있는 제품을 함께 추천할 수 있도록 모델 구성

## 평균구매주기 계산

In [34]:
memberwise_sales

,회원번호,주문일시,회원상태,구매수량,구매금액,배송시작일,배송완료일,사용 적립금,사용 포인트 네이버,주문취소여부,주문시간,제품번호
0,18764160,2021-01-03,2,2,2,2,2,2,2,0,2,2
1,18764160,2021-01-23,3,3,3,3,3,3,3,0,3,3
2,18764160,2021-03-30,2,2,2,2,2,2,2,0,2,2
3,18764160,2021-04-11,1,1,1,1,1,1,1,0,1,1
4,18764160,2021-04-14,1,1,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
193289,1670620864,2021-04-27,1,1,1,1,1,1,1,0,1,1
193290,1670620864,2021-04-28,8,8,8,8,8,8,8,0,8,8
193291,1670620864,2021-04-30,2,2,2,2,2,2,2,0,2,2
193292,1670620864,2021-06-02,2,2,2,2,2,2,2,0,2,2


In [62]:
def ppp(active_member, memberwise_sales):
    # 정상회원 멤버 리스트
    member_list = memberwise_sales.loc[memberwise_sales['회원번호'].isin(active_member)]
    
    a = []  # 회원번호 저장하는 리스트
    b = []  # 회원별 평균 구매 주기 저장하는 리스트
    
    # 멤버 한 명씩 주문 일시를 살펴볼 때,
    for member in tqdm(member_list['회원번호']):
        target_member_sales = memberwise_sales[memberwise_sales['회원번호'] == member].sort_values(by='주문일시')
        
        # 특정 타겟 멤버의 주문 내역이 2개 이상일 때만 구매 주기 계산
        if len(target_member_sales) > 1:
            purchase_period = []
            for i in range(len(target_member_sales) - 1):
                purchase_period.append(target_member_sales['주문일시'].iloc[i+1] - target_member_sales['주문일시'].iloc[i])
            
            average_period = np.mean(purchase_period)
        else:
            average_period = np.nan  # 주문이 하나만 있는 경우 NaN으로 설정

        a.append(member)
        b.append(average_period)

    list_df = pd.DataFrame({'회원번호': a, '평균구매주기': b})
    
    return list_df

In [165]:
#active_member = df_member[df_member['회원상태'] == '정상회원']['회원번호'].unique()
active_member = df_member[df_member['회원상태'] == '정상회원']

In [166]:
#실행
ppp_result = ppp(active_member['회원번호'].unique(), memberwise_sales)
display(ppp_result)

  0%|          | 697/191575 [00:05<25:18, 125.74it/s]


KeyboardInterrupt: 

In [167]:
active_member = pd.merge(active_member, ppp_result.drop_duplicates(subset='회원번호'))
active_member
# 평균구매주기가 긴 회원은 오랜만에 "돌아온" 사람
# 평균구매주기가 NaT인 회원은 유령회원이거나 신규가입고객이라 구매일시가 하루밖에 없는 회원들
# 신규회원과 유령회원을 구분하기 위해서 최초 주문일시가 전체 기간의 80% 시점 이전 주문 고객만 따로 봐야 할듯

,회원번호,회원상태,성별,나이,등록카드,결혼,구독여부,주소지,세부주소지,클러스터,평균구매주기
0,18764160,정상회원,여,68,농협중앙회,기혼,False,서울특별시,성동구,0,19 days 17:08:34.285714285
1,18792000,정상회원,남,83,연결앱결제,NaN,False,강원도,강릉시,0,NaT
2,18942336,정상회원,여,39,신한은행,기혼,False,인천광역시,중구,1,23 days 16:00:00
3,18949760,정상회원,여,73,기업은행,NaN,nan,강원도,홍천군,0,NaT
4,19391488,정상회원,여,52,연결앱결제,기혼,False,대전광역시,중구,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...
12375,1644772352,정상회원,여,50,국민카드,기혼,False,강원도,삼척시,0,NaT
12376,1644957952,정상회원,여,34,신한은행,미혼,False,부산광역시,중구,3,NaT
12377,1645334720,정상회원,남,24,국민카드,미혼,False,전라남도,여수시,3,NaT
12378,1670606016,정상회원,남,19,하나은행,미혼,nan,경상남도,밀양시,3,38 days 14:24:00


## 과거 구매 목록 기반 상품 추천

In [85]:
# 회원번호를 입력하면 평균구매주기와 자주구매한 상품, 추천할 상품을 리턴하는 함수
def recommendation(active_member, member, df_sales, df_product):
    
    target_active_member = active_member[active_member['평균구매주기'].notnull()]

    a = []  # 회원번호 저장하는 리스트
    b = []  # 회원별 평균 구매 주기 저장하는 리스트
    c = []  # 해당회원이 자주 구매한 상품 리스트
    d = []  # 추천할 상품 리스트
    item_c = [] # 자주구매한 상품명 저장하는 리스트
    item_d = [] # 추천할 상품명 저장하는 리스트

    list_df = pd.DataFrame()
    
    # 유령고객/신규고객 이 아닌 데이터가 충분히 있는 고객에 대해
    if member in (target_active_member['회원번호'].values):
        member_cond = (target_active_member['회원번호'] == member)

        # 해당 고객의 구매 내역
        member_sales = df_sales.loc[df_sales['회원번호'] == member]

        # 해당 회원이 자주 구매한 상위 5개(최대) 항목 출력
        if len(member_sales['제품번호'].value_counts()) >= 5:
            top5_item = member_sales['제품번호'].value_counts().head(5)
            
            c.append(top5_item)

            # top5_item을 리스트로 변환하여 랜덤 샘플링
            d.append(random.sample(top5_item.index.tolist(), 3))  # 상위 5개 항목에서 랜덤으로 3개 선택해서 추천

        else:
            top_item = member_sales['제품번호'].value_counts()
            
            c.append(top_item)

            d.append(top_item.index.tolist())  # 주문데이터가 많이 없으므로(5건 미만) 구매리스트와 추천리스트는 동일하게 구성
        
        a.append(member)  # 회원번호 append
        b.append(target_active_member.loc[member_cond]['평균구매주기'].values[0].astype('timedelta64[D]').item())  # 평균구매주기 append

        for i in range(len(c)):
            item_cond = (df_product['제품번호'].isin(c[i].index))  # c[i]의 인덱스를 사용하여 조건 생성
            item_c.append(df_product.loc[item_cond, '물품명'].values)  # 제품번호에 해당하는 물품명 저장

        
        for i in range(len(d)):
            for product_number in d[i]:  # d[i]의 각 제품 번호에 대해 반복
                item_cond = (df_product['제품번호'] == product_number)  # 각 제품 번호에 대해 조건 생성
                item_d.append(df_product.loc[item_cond, '물품명'].values[0])  # 제품번호에 해당하는 물품명 저장
   
        list_df = pd.DataFrame({'회원번호': a,
                                '평균구매주기': b,
                                '자주구매한제품': item_c,
                                '추천상품': [item_d]})

    else:
        print(f"{member}의 구매 데이터가 충분하지 않습니다.(유령회원/신규회원)")
    
    return list_df        

#recommend_result = recommendation(active_member, int(input("고객번호 입력: ")), df_sales, df_product)
recommend_result = recommendation(active_member, 18764160, df_sales, df_product)

recommend_result


고객번호 입력:  18764160


,회원번호,평균구매주기,자주구매한제품,추천상품
0,18764160,19 days,"[면생리대, 물사랑 어린이치약, 물사랑치약, 키토산비누, 흑염소진액]","[면생리대, 흑염소진액, 물사랑치약]"


In [87]:
for i in range(len(active_member.head(5))):
    recommend_full_result = recommendation(active_member,
                                           active_member['회원번호'].iloc[i],
                                           df_sales,
                                           df_product)
    display(recommend_full_result)

,회원번호,평균구매주기,자주구매한제품,추천상품
0,18764160,19 days,"[면생리대, 물사랑 어린이치약, 물사랑치약, 키토산비누, 흑염소진액]","[키토산비누, 물사랑치약, 면생리대]"


18792000의 구매 데이터가 충분하지 않습니다.(유령회원/신규회원)


""


,회원번호,평균구매주기,자주구매한제품,추천상품
0,18942336,23 days,"[느타리버섯, 백숙용통닭, 복숭아, 수피아 내츄럴 수분에센셜크림, 콩나물]","[백숙용통닭, 복숭아, 콩나물]"


18949760의 구매 데이터가 충분하지 않습니다.(유령회원/신규회원)


""


19391488의 구매 데이터가 충분하지 않습니다.(유령회원/신규회원)


""


In [88]:
active_member.head(5)

,회원번호,회원상태,성별,나이,등록카드,결혼,구독여부,주소지,세부주소지,평균구매주기
0,18764160,정상회원,여,68,농협중앙회,기혼,False,서울특별시,성동구,19 days 17:08:34.285714285
1,18792000,정상회원,남,83,연결앱결제,NaN,False,강원도,강릉시,NaT
2,18942336,정상회원,여,39,신한은행,기혼,False,인천광역시,중구,23 days 16:00:00
3,18949760,정상회원,여,73,기업은행,NaN,NaN,강원도,홍천군,NaT
4,19391488,정상회원,여,52,연결앱결제,기혼,False,대전광역시,중구,NaT


In [89]:
def p_id_to_name(p_id_list, df_product):
    name = [] # 제품명 리스트
    
    for i in range(len(p_id_list)):
        item_cond = (df_product['제품번호'] == p_id_list[i])  # 각 제품 번호에 대해 조건 생성
        name.append(df_product.loc[item_cond, '물품명'].values[0])  # 제품번호에 해당하는 물품명 저장

    return name

## 유사 고객 기반 상품 추천
- 유사도 계산: 코사인 유사도 등을 사용하여 각 고객 간의 유사도를 산출

- 
추천 대상 제품: 예를 들어, 고객 A와 가장 유사한 상위 N명의 고객(B 그룹)을 찾고, 이들이 구매한 제품 중 고객 A가 구매하지 않은 제품을 추출하여 추천

In [90]:
# 피벗 테이블 생성: 행은 고객(회원번호), 열은 제품(제품번호), 값은 구매 횟수 (없으면 0)
customer_product = df_sales.pivot_table(
    index='회원번호',
    columns='제품번호',
    values='구매수량',   # 구매수량이 있거나 단순히 구매 여부를 나타내기 위해 사용
    aggfunc='sum',
    fill_value=0
)

# 간단하게 이진 행렬로 변환 (1: 구매 경험 있음, 0: 없음)
customer_product_binary = (customer_product > 0).astype(int)
customer_product_binary

제품번호,100021000V2_761,100021005V2_762,100021006V2_763,100021007V2_764,100021009V2_765,10002100V2_77,100021013V2_766,100021014V2_767,100021016V2_768,100021019V2_769,...,10002989V2_754,1000298V2_75,10002990V2_755,10002991V2_756,10002992V2_757,10002993V2_758,10002998V2_759,10002999V2_760,1000299V2_76,100029V2_7
회원번호,,,,,,,,,,,,,,,,,,,,,
18764160,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18792000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18942336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
18949760,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19391488,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1644957952,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1645334720,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1670400000,1,1,1,0,1,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0


In [91]:
from sklearn.metrics.pairwise import cosine_similarity

# 고객 간 코사인 유사도 계산
similarity_matrix = cosine_similarity(customer_product_binary)
similarity_df = pd.DataFrame(similarity_matrix, 
                             index=customer_product_binary.index, 
                             columns=customer_product_binary.index)

def recommend_products(customer_id, top_similar=5, top_n=5):
    """
    - customer_id: 추천 대상 고객 ID
    - top_similar: 유사 고객 상위 몇 명을 고려할지
    - top_n: 추천할 제품 수
    """
    # 1. 대상 고객과 유사도가 높은 고객 추출 (자기 자신 제외)
    similar_customers = similarity_df.loc[customer_id].sort_values(ascending=False)
    similar_customers = similar_customers.drop(customer_id).head(top_similar).index.tolist()
    
    # 2. 대상 고객이 이미 구매한 제품 목록
    customer_products = set(customer_product_binary.columns[customer_product_binary.loc[customer_id] > 0])
    
    # 3. 유사 고객들이 구매한 제품들의 구매 빈도 합산
    similar_group = customer_product_binary.loc[similar_customers]
    product_scores = similar_group.sum().sort_values(ascending=False)
    
    # 4. 대상 고객이 구매하지 않은 제품 중 상위 top_n 제품 추천
    recommended_products = [prod for prod in product_scores.index if prod not in customer_products]
    return recommended_products[:top_n]

# 예시: 고객 A (예를 들어, 고객 ID가 customer_cycle의 첫 번째 회원번호)를 대상으로 추천
example_customer = 18764160
print("고객 {}의 추천 제품:".format(example_customer), p_id_to_name(recommend_products(example_customer), df_product))

고객 18764160의 추천 제품: ['냉동모듬찰떡', '액상세제', '당면', '창포샴푸비누', '흑토마토']


## 정기구독 목록 구성: 과거 구매이력 + 유사 고객 기반 최종 구독목록 추천

In [97]:
def subscription_recommendation(customer_id, top_n=5):
    """
    - 고객의 과거 구매 내역에서 자주 구매한 제품과
      유사 고객 추천을 결합하여 정기구독 추천 목록 작성
    """
    # 1. 고객의 구매 내역 (빈도수 기준 상위 3개 제품)
    customer_history = df_sales[df_sales['회원번호'] == customer_id]
    frequent_products = recommendation(active_member, customer_id, df_sales, df_product)
    
    # 2. 유사 고객 기반 추천 제품 (위에서 정의한 recommend_products 함수 사용)
    recommended = p_id_to_name(recommend_products(customer_id, top_similar=5, top_n=top_n), df_product)
    
    # 3. 두 목록을 합치고 중복 제거
    # frequent_products['추천상품']에서 리스트를 추출하고 평탄화
    frequent_product_list = frequent_products['추천상품'].values[0] if not frequent_products.empty else []
    
    subscription_list = list(set(frequent_product_list + recommended))
    return subscription_list

# 예시: 고객 A의 정기구독 추천 목록
sub_list = subscription_recommendation(18764160)
print("고객 {}의 정기구독 추천 목록:".format(18764160), sub_list)


고객 18764160의 정기구독 추천 목록: ['냉동모듬찰떡', '액상세제', '흑토마토', '키토산비누', '흑염소진액', '창포샴푸비누', '당면', '물사랑 어린이치약']


# Feature 4.
- 주문시간, 거주지, 구매제품종류 등을 input으로 입력받으면 배송지연 여부를 예측하는 모델 생성. 
- 고객 군집, 주문특성(주문시간, 거주지, 구매상품 종류 등)과 배송기간 사이의 관계를 회귀분석(다른 방법을 사용해도 무방)을 이용해 파악.
-  배송 지연된 경험을 한 고객들만 따로 뽑아서 고객이탈률, 구독유무를 계산(가설검정: 배송지연발생유무(혹은 빈도)가 이탈률, 구독유무와 상관관계가 있다)

In [268]:
# 유령회원 판별을 위한 최신주문일시 칼럼 추가
temp = memberwise_sales.pivot_table(index='회원번호', values='주문일시', aggfunc='max').rename(columns={'주문일시':'최신주문일시'}).reset_index()
active_member = pd.merge(active_member, temp)

# 배송 지연 비율 구하기 위한 칼럼 추가
temp2 = df_sales.pivot_table(index='회원번호', values='주문일시', aggfunc='count').rename(columns={'주문일시': '주문횟수'}).reset_index()
active_member = pd.merge(active_member,temp2)

In [269]:
# 배송 지연 경험이 한 번이라도 있는 고객
late_exp = deli_result['delivery_count'] > 0

# 탈퇴 회원 구하기 위한 임시 df
late_exp_member = pd.merge(df_member, deli_result.loc[late_exp])

# 유령회원 구하기 위한 임시 df
late_exp_member2 = pd.merge(active_member, deli_result.loc[late_exp])
late_exp_member2['배송지연비율'] = late_exp_member2['delivery_count'] / late_exp_member2['주문횟수']

# 절대적인 수치로 배송지연을 계산 시 주문 횟수가 많은 경우 정확한 비교가 불가능하므로 비율로 따짐.
# 배송 지연 경험이 10% 초과인 경우
late_exp_member2 = late_exp_member2.loc[late_exp_member2['배송지연비율'] > 0.1]
late_exp_member2

,회원번호,회원상태,성별,나이,등록카드,결혼,구독여부,주소지,세부주소지,클러스터,평균구매주기,최신주문일시,주문횟수,delivery_count,배송지연비율
0,18764160,정상회원,여,68,농협중앙회,기혼,False,서울특별시,성동구,0,19 days 17:08:34.285714285,2021-10-06,36,4,0.111111
9,22259008,정상회원,여,46,신한은행,기혼,nan,강원도,정선군,1,29 days 21:00:00,2021-10-30,33,4,0.121212
14,25145088,정상회원,여,47,연결앱결제,NaN,False,경상남도,김해시,2,24 days 00:00:00,2021-03-17,25,3,0.120000
21,29868608,정상회원,여,57,농협중앙회,NaN,False,경상북도,경주시,0,70 days 16:00:00,2021-09-05,5,2,0.400000
25,37253632,정상회원,여,43,BC카드,기혼,False,세종특별자치시,연서면,1,NaT,2021-02-12,5,1,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6834,1641750784,정상회원,여,45,하나은행,기혼,True,서울특별시,영등포구,1,NaT,2021-10-18,7,1,0.142857
6835,1643209600,정상회원,여,34,롯데카드,NaN,False,서울특별시,강남구,2,4 days 00:00:00,2021-10-28,5,1,0.200000
6836,1644557056,정상회원,여,55,롯데카드,기혼,False,강원도,강릉시,0,1 days 00:00:00,2021-10-30,3,1,0.333333
6837,1644705536,정상회원,여,66,국민은행,미혼,True,경기도,평택시,0,NaT,2021-10-30,5,1,0.200000


### 배송 지연 경험률이 n% 초과인 고객 중 고객 이탈률(유령회원 + 탈퇴)

In [270]:
# 배송 지연 경험률이 10% 초과인 고객 중 유령회원
late_ghost_cond = late_exp_member2['최신주문일시'] < datetime.datetime(2021, 8, 30)

late_ghost = late_exp_member2.loc[late_ghost_cond]

# 배송 지연 경험률이 10% 초과인 고객 중 탈퇴회원
late_passive = late_exp_member.loc[late_exp_member['회원상태'] != '정상회원']


(len(late_ghost) + len(late_passive)) / len(late_exp_member2) * 100

53.480753480753485

### 전체 고객 중 고객 이탈률(유령회원 + 탈퇴)

In [271]:
# 전체 고객 중 유령회원
ghost_cond = (active_member['최신주문일시'] < datetime.datetime(2021, 8, 30))

ghost = active_member.loc[ghost_cond]

# 전체 고객 중 탈퇴 회원
passive = df_member.loc[df_member['회원상태'] != '정상회원']


(len(ghost)+ len(passive)) / len(df_member) * 100

36.68261562998405

### 배송 지연 경험률이 n% 초과인 고객 중 구독률

In [277]:
len(late_exp_member2.loc[late_exp_member2['구독여부'] == 'True']) / len(late_exp_member2)

0.1294021294021294

In [282]:
# 전체 고객의 구독률을 13.8~% 였음. 크게 차이가 나지는 않는 것으로 보이긴 하는데 가설검정 해보기

### 가설 검정

In [284]:
import statsmodels.api as sm

In [286]:
# 배송 지연 경험이 있는 고객의 이탈률
n_late_churn = len(late_ghost) + len(late_passive)  # 배송 지연 경험이 있는 이탈고객 수
n_late_total = len(late_exp_member2)  # 배송 지연 경험이 있는 전체 고객 수

# 전체 고객 이탈률
n_churn = len(ghost)+ len(passive)  # 전체 이탈고객 수
n_total = len(df_member)  # 전체 고객 수

# 비율 계산
p1 = n_late_churn / n_late_total  # 배송 지연 경험이 있는 고객의 이탈률
p2 = n_churn / n_total  # 전체 고객 이탈률

# Z-검정 수행
count = np.array([n_late_churn, n_churn])
nobs = np.array([n_late_total, n_total])

z_score, p_value = sm.stats.proportions_ztest(count, nobs)

# 결과 출력
print(f"Z-검정 통계량: {z_score}")
print(f"P-값: {p_value}")

# 유의수준 설정 (예: 0.05)
alpha = 0.05
if p_value < alpha:
    print("귀무가설을 기각합니다. 배송 지연 비율은 고객 이탈과 상관이 있습니다.")
else:
    print("귀무가설을 기각할 수 없습니다. 배송 지연 비율은 고객 이탈과 상관이 없습니다.")


Z-검정 통계량: 11.533876517804034
P-값: 8.904203639554603e-31
귀무가설을 기각합니다. 배송 지연 비율은 고객 이탈과 상관이 있습니다.


In [287]:
# 배송 지연 경험이 있는 고객의 구독률
n_late_sub = len(late_exp_member2.loc[late_exp_member2['구독여부'] == 'True'])  # 배송 지연 경험이 있는 구독 수

# 전체 고객  구독률
n_sub = len(df_member.loc[df_member['구독여부'] == 'True'])  # 전체 이탈고객 수

# 비율 계산
p1 = n_late_sub / n_late_total  # 배송 지연 경험이 있는 고객의 이탈률
p2 = n_sub / n_total  # 전체 고객 중 유령회원의 이탈률

# Z-검정 수행
count = np.array([n_late_sub, n_sub])
nobs = np.array([n_late_total, n_total])

z_score, p_value = sm.stats.proportions_ztest(count, nobs)

# 결과 출력
print(f"Z-검정 통계량: {z_score}")
print(f"P-값: {p_value}")

# 유의수준 설정 (예: 0.05)
alpha = 0.05
if p_value < alpha:
    print("귀무가설을 기각합니다. 배송 지연 비율은 고객 이탈과 상관이 있습니다.")
else:
    print("귀무가설을 기각할 수 없습니다. 배송 지연 비율은 고객 이탈과 상관이 없습니다.")


Z-검정 통계량: -1.0022589655809622
P-값: 0.31621853554386536
귀무가설을 기각할 수 없습니다. 배송 지연 비율은 고객 이탈과 상관이 없습니다.
